### Deep Learning Classifier (Melinda Xiao-Devins)

Implement deep learning modle


In [25]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import os

In [26]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
#from tensorflow.python.layers.core import Dense
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
 warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
 print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [27]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [28]:
# load dataset
#df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")
df = pd.read_csv('./data/ny_hmda_2015_robust.csv', low_memory=False, header=0, delimiter=",")

#print(dataframe.loc[:,:])
num_rows = df.shape[0]
num_col = df.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))

X = np.array(df.drop(['action_taken'],1)) 
Y = np.array(df['action_taken'])

#Split into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Total number of records: 376516
Toatl numver of features: 38


In [32]:
# Create a fully connected network with 4 layers
model = Sequential()

#input layer, it has 256 neurons, it must have right number of inputs, which is the number of features
model.add(Dense(256, input_dim=num_col-1, kernel_initializer='normal', activation='relu'))

# hideen layer has 128 neurons
model.add(Dense(128, kernel_initializer='normal', activation='relu'))

# hideen layer has 64 neurons
model.add(Dense(128, kernel_initializer='normal', activation='relu'))

# hideen layer has 64 neurons
model.add(Dense(64, kernel_initializer='normal', activation='relu'))

# output layer has 1 neuron to predict
# Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [33]:
#Compile model

# loss function: logarithmic loss, which is binary_crossentropy for binary classification
# use 'adam' optimizer for gradient descent algorithm 
# collect accuracy during training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# Train the model
# epochs: a fixed number of iterations through the dataset
# batch size: the number of instances that are evaluated before a weight update in the network is performed 
model.fit(X_train, Y_train, epochs = 500, batch_size=100)

Epoch 1/100
301212/301212 [==============================] - 14s 46us/step - loss: 0.3389 - acc: 0.8264
Epoch 2/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3285 - acc: 0.8342
Epoch 3/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3265 - acc: 0.8353
Epoch 4/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3249 - acc: 0.8365
Epoch 5/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3238 - acc: 0.8372
Epoch 6/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3232 - acc: 0.8377
Epoch 7/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3225 - acc: 0.8380
Epoch 8/100
301212/301212 [==============================] - 13s 43us/step - loss: 0.3219 - acc: 0.8386
Epoch 9/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3215 - acc: 0.8387
Epoch 10/100
301212/301212 [==============================] - 13

301212/301212 [==============================] - 13s 43us/step - loss: 0.3014 - acc: 0.8525
Epoch 80/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3015 - acc: 0.8522
Epoch 81/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3014 - acc: 0.8522
Epoch 82/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3011 - acc: 0.8526
Epoch 83/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3013 - acc: 0.8525
Epoch 84/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3006 - acc: 0.8526
Epoch 85/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3003 - acc: 0.8533
Epoch 86/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.3002 - acc: 0.8535
Epoch 87/100
301212/301212 [==============================] - 13s 44us/step - loss: 0.2999 - acc: 0.8535
Epoch 88/100
301212/301212 [==============================] - 13s 44

In [ ]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))


## Save Trained Model
Save the trained model to disk

In [ ]:
 
# serialize model to JSON
model_json = model.to_json()
with open("model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model/model.h5")
print("Saved model to disk")

## Load Saved Model
Load the saved model, and used it. It saves training time

In [ ]:
# load json and create model
json_file = open('model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model/model.h5")
print("Loaded model from disk")
 
print("Use loaded model to predict")
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])



In [ ]:
score = loaded_model.evaluate(X, Y, verbose=0)
print("From saved model,  %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))



In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))
